In [1]:
import pandas as pd

# Load the datasets
database_file = pd.read_excel('database_file.xlsx')
source_data = pd.read_excel('Source_data.xlsx')
card_market_mapping = pd.read_excel('Card market mapping.xlsx')


# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of each dataset for initial inspection
database_file_head = database_file.head()
source_data_head = source_data.head()


In [2]:
database_file_head

,MKCID,Card Market Code,Business Process Date,Billed Amount,Billed Currency
0,62651170,17,"September 30, 2023",3546.73,MYR
1,16070738,360,"March 22, 2023",2747.72,SAR
2,16967315,777,"February 20, 2024",4055.35,TWD
3,97921681,752,2023-09-28,1918.43,USD
4,72703764,17,27-Jul-2022,9798.17,MYR


In [3]:
 source_data_head

,Card Region,Card Market,MKCID,Client Organisation,Business Process Date,Billed Currency,Net Billed Amount
0,Asia/Pacific,Malaysia,62651170,Parker Group,23-Nov-2022,MYR,3546.73
1,Mideast,Saudi Arabia,16070738,"Young, Ayers and Aguirre",02/10/2023,SAR,2747.72
2,Asia/Pacific,Taiwan,16967315,"Taylor, Mason and Rivera",05-27-2023,TWD,4055.35
3,Latin America,Argentina,97921681,Simpson Group,01/06/2024,USD,2167.15
4,Asia/Pacific,Malaysia,72703764,"Hanna, Barajas and Long",21-Aug-2023,MYR,9798.17


In [4]:
 card_market_mapping.head()

,Card Market,Card Market Code
0,Malaysia,17
1,Saudi Arabia,360
2,Taiwan,777
3,Argentina,752
4,Malaysia,17


Data Formatting

In [5]:
# Standardizing column names for easier comparison
database_file.columns = database_file.columns.str.replace(' ', '_').str.lower()
source_data.columns = source_data.columns.str.replace(' ', '_').str.lower()

In [6]:
# Standardizing date formats
database_file['business_process_date'] = pd.to_datetime(database_file['business_process_date'], errors='coerce')
source_data['business_process_date'] = pd.to_datetime(source_data['business_process_date'], errors='coerce')

C:\Users\Swarali Bavalekar\AppData\Local\Temp\ipykernel_29096\3512894788.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  database_file['business_process_date'] = pd.to_datetime(database_file['business_process_date'], errors='coerce')
C:\Users\Swarali Bavalekar\AppData\Local\Temp\ipykernel_29096\3512894788.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  source_data['business_process_date'] = pd.to_datetime(source_data['business_process_date'], errors='coerce')


In [7]:
# Renaming columns for a clear join
database_file.rename(columns={'card_market_code': 'card_market', 'billed_amount': 'database_billed_amount'}, inplace=True)
source_data.rename(columns={'net_billed_amount': 'source_billed_amount'}, inplace=True)

In [8]:
# Inspect data types of both datasets
database_file_dtypes = database_file.dtypes
source_data_dtypes = source_data.dtypes

database_file_dtypes, source_data_dtypes

(mkcid                              int64
 card_market                        int64
 business_process_date     datetime64[ns]
 database_billed_amount           float64
 billed_currency                   object
 dtype: object,
 card_region                      object
 card_market                      object
 mkcid                             int64
 client_organisation              object
 business_process_date    datetime64[ns]
 billed_currency                  object
 source_billed_amount            float64
 dtype: object)

In [9]:
# Check for non-numeric values in the card_market column of source_data
non_numeric_card_market = source_data[~source_data['card_market'].str.isdigit()]
non_numeric_card_market


,card_region,card_market,mkcid,client_organisation,business_process_date,billed_currency,source_billed_amount
0,Asia/Pacific,Malaysia,62651170,Parker Group,2022-11-23,MYR,3546.73
1,Mideast,Saudi Arabia,16070738,"Young, Ayers and Aguirre",2023-02-10,SAR,2747.72
2,Asia/Pacific,Taiwan,16967315,"Taylor, Mason and Rivera",2023-05-27,TWD,4055.35
3,Latin America,Argentina,97921681,Simpson Group,2024-01-06,USD,2167.15
4,Asia/Pacific,Malaysia,72703764,"Hanna, Barajas and Long",2023-08-21,MYR,9798.17
...,...,...,...,...,...,...,...
99995,Mideast,UAE,59176224,"James, Saunders and Franco",2023-01-14,AED,5323.64
99996,Asia/Pacific,Taiwan,63540547,Hood-White,2023-07-16,TWD,1378.76
99997,Asia/Pacific,Japan,35496683,Edwards-Russell,2024-02-24,USD,4083.60
99998,Asia/Pacific,Indonesia,77626533,James-Stevenson,2024-07-07,IDR,4450.57


In [10]:
# Create a dictionary for the mapping
market_code_mapping = dict(zip(card_market_mapping['Card Market'], card_market_mapping['Card Market Code']))
    
# Replace card_market names in source_data with their corresponding codes
source_data['card_market'] = source_data['card_market'].map(market_code_mapping)

In [11]:
# Check for non-numeric values and handle them
if source_data['card_market'].isna().any():
    unmapped_markets = source_data[source_data['card_market'].isna()]['card_market'].unique()
    raise ValueError(f"Some card markets in source data could not be mapped to numeric codes: {unmapped_markets}")
    

In [12]:
# Convert card_market in source_data to int64 for consistency
source_data['card_market'] = source_data['card_market'].astype('int64')

In [13]:
# Ensure all merge columns have the same data type in both dataframes
database_file['card_market'] = database_file['card_market'].astype('int64')
database_file['mkcid'] = database_file['mkcid'].astype('int64')
source_data['mkcid'] = source_data['mkcid'].astype('int64')

In [14]:
database_file, source_data

(          mkcid  card_market business_process_date  database_billed_amount  \
 0      62651170           17            2023-09-30                 3546.73   
 1      16070738          360            2023-03-22                 2747.72   
 2      16967315          777            2024-02-20                 4055.35   
 3      97921681          752            2023-09-28                 1918.43   
 4      72703764           17            2022-07-27                 9798.17   
 ...         ...          ...                   ...                     ...   
 99995  59176224          972            2023-03-17                 5092.14   
 99996  63540547          777            2023-07-18                 1378.76   
 99997  35496683          314            2022-09-30                 4507.96   
 99998  77626533          985            2023-10-26                 4450.57   
 99999  33801110          135            2022-10-12                 7913.55   
 
       billed_currency  
 0                 MYR  


In [15]:
merged_data = pd.merge(database_file, source_data, on=['mkcid', 'business_process_date', 'billed_currency', 'card_market'], how='outer', indicator=True)

In [16]:
merged_data.head()

,mkcid,card_market,business_process_date,database_billed_amount,billed_currency,card_region,client_organisation,source_billed_amount,_merge
0,62651170,17,2023-09-30,3546.73,MYR,Asia/Pacific,Parker Group,2963.00,both
1,62651170,17,2023-09-30,3546.73,MYR,Asia/Pacific,Parker Group,9866.43,both
2,16070738,360,2023-03-22,2747.72,SAR,Mideast,"Young, Ayers and Aguirre",2084.53,both
3,16070738,360,2023-03-22,2747.72,SAR,Mideast,"Young, Ayers and Aguirre",1949.60,both
4,16070738,360,2023-03-22,2747.72,SAR,Mideast,"Young, Ayers and Aguirre",6949.48,both


In [17]:
discrepancies = merged_data[
        (merged_data['_merge'] == 'both') & 
        (merged_data['database_billed_amount'] != merged_data['source_billed_amount'])
    ]

In [18]:
 # Identify rows that are only in database_file or only in source_data
missing_in_database = merged_data[merged_data['_merge'] == 'right_only']
missing_in_source = merged_data[merged_data['_merge'] == 'left_only']
    

In [20]:
# Generate a summary of discrepancies by market and date
discrepancy_summary = discrepancies.groupby(['card_market', 'business_process_date']).size().reset_index(name='discrepancy_count')
discrepancy_summary    

,card_market,business_process_date,discrepancy_count
0,17,2022-03-09,1
1,17,2022-07-27,4
2,17,2022-07-28,6
3,17,2022-07-30,4
4,17,2022-07-31,3
...,...,...,...
20539,985,2024-07-22,2
20540,985,2024-07-23,3
20541,985,2024-07-24,5
20542,985,2024-07-26,3


In [24]:
import pandas as pd

def load_data(database_path, source_data_path, mapping_path):
    # Load the datasets
    database_file = pd.read_excel(database_path)
    source_data = pd.read_excel(source_data_path)
    card_market_mapping = pd.read_excel(mapping_path)
    
    # Standardize column names for easier comparison
    database_file.columns = database_file.columns.str.replace(' ', '_').str.lower()
    source_data.columns = source_data.columns.str.replace(' ', '_').str.lower()
    
    # Standardize date formats
    database_file['business_process_date'] = pd.to_datetime(database_file['business_process_date'], errors='coerce')
    source_data['business_process_date'] = pd.to_datetime(source_data['business_process_date'], errors='coerce')
    
    # Rename columns for a clear join
    database_file.rename(columns={'card_market_code': 'card_market', 'billed_amount': 'database_billed_amount'}, inplace=True)
    source_data.rename(columns={'net_billed_amount': 'source_billed_amount'}, inplace=True)
    
    return database_file, source_data, card_market_mapping

def prepare_data(database_file, source_data, card_market_mapping):
    # Create a dictionary for the mapping
    market_code_mapping = dict(zip(card_market_mapping['Card Market'], card_market_mapping['Card Market Code']))
    
    # Replace card_market names in source_data with their corresponding codes
    source_data['card_market'] = source_data['card_market'].map(market_code_mapping)
    
    # Convert card_market in source_data to int64 after replacing
    source_data['card_market'] = source_data['card_market'].astype('int64')
    
    return database_file, source_data

def merge_data(database_file, source_data):
    # Perform the merge
    merged_data = pd.merge(database_file, source_data, on=['mkcid', 'business_process_date', 'billed_currency', 'card_market'], how='outer', indicator=True)
    return merged_data

def identify_discrepancies(merged_data):
    # Identify rows with discrepancies in billed amounts
    discrepancies = merged_data[
        (merged_data['_merge'] == 'both') & 
        (merged_data['database_billed_amount'] != merged_data['source_billed_amount'])
    ]
    
    # Identify rows that are only in database_file or only in source_data
    missing_in_database = merged_data[merged_data['_merge'] == 'right_only']
    missing_in_source = merged_data[merged_data['_merge'] == 'left_only']
    
    # Generate a summary of discrepancies by market and date
    discrepancy_summary = discrepancies.groupby(['card_market', 'business_process_date']).size().reset_index(name='discrepancy_count')
    
    return discrepancy_summary, discrepancies, missing_in_database, missing_in_source

def save_reports(output_file, discrepancy_summary, discrepancies, missing_in_database, missing_in_source):
    with pd.ExcelWriter(output_file) as writer:
        # Write the summary of discrepancies
        discrepancy_summary.to_excel(writer, sheet_name='Discrepancy Summary', index=False)
        
        # Write the detailed discrepancies
        discrepancies.to_excel(writer, sheet_name='Discrepancies', index=False)
        
        # Write the missing transactions
        missing_in_database.to_excel(writer, sheet_name='Missing in Database', index=False)
        missing_in_source.to_excel(writer, sheet_name='Missing in Source', index=False)
        
        # Create separate sheets for each market for detailed analysis
        for market in discrepancies['card_market'].unique():
            market_data = discrepancies[discrepancies['card_market'] == market]
            market_name = f'Market_{market}'
            market_data.to_excel(writer, sheet_name=market_name, index=False)

def main(database_path, source_data_path, mapping_path, output_file):
    database_file, source_data, card_market_mapping = load_data(database_path, source_data_path, mapping_path)
    database_file, source_data = prepare_data(database_file, source_data, card_market_mapping)
    merged_data = merge_data(database_file, source_data)
    discrepancy_summary, discrepancies, missing_in_database, missing_in_source = identify_discrepancies(merged_data)
    save_reports(output_file, discrepancy_summary, discrepancies, missing_in_database, missing_in_source)

# Paths to the input files
database_path = 'C:/Users/Swarali Bavalekar/Documents/Data Project/Validation project.zip/Validation project/database_file.xlsx'
source_data_path = 'C:/Users/Swarali Bavalekar/Documents/Data Project/Validation project.zip/Validation project/source_data.xlsx'
mapping_path = 'C:/Users/Swarali Bavalekar/Documents/Data Project/Validation project.zip/Validation project/card_market_mapping.xlsx'
output_file = 'C:/Users/Swarali Bavalekar/Documents/Data Project/Validation project.zip/Validation project/discrepancy_report.xlsx'

# Run the main function
main(database_path, source_data_path, mapping_path, output_file)


C:\Users\Swarali Bavalekar\AppData\Local\Temp\ipykernel_29096\2825645530.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  database_file['business_process_date'] = pd.to_datetime(database_file['business_process_date'], errors='coerce')
C:\Users\Swarali Bavalekar\AppData\Local\Temp\ipykernel_29096\2825645530.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  source_data['business_process_date'] = pd.to_datetime(source_data['business_process_date'], errors='coerce')
